In [42]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import xgboost as xgb
from numpy import linalg
import socket
from scipy.stats import zscore, mode
import json
import time

In [37]:
def pm(angle):
    if angle > np.pi:
        while angle > np.pi:
            angle -= 2*np.pi
    if angle < -np.pi:
        while angle < -np.pi:
            angle += 2*np.pi
    return angle

pm = np.vectorize(pm)


wave_dist= (299_792_458/(2.4e6+270e3))/50
#phase_diff - 29= np.sin(np.deg2rad(angles))*2*np.pi/wave_dist
aoa = lambda pds: np.rad2deg( np.arcsin(np.clip(pm(pds[1:] - pds[:-1] -np.deg2rad(2*99)) *wave_dist/(2*np.pi),-1,1))) 

# Classifier

In [70]:
model = xgb.XGBClassifier()
model.load_model("./xgboost/tw2/pd/xgb_classifier.json")

with open("./xgboost/tw2/pd/dicts.json") as file:
    dicts = json.load(file)

moving_avarage_model = [0]*8
moving_avarage_vannila = [0]*8

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
    
def on_message(client, userdata, msg):
    
    data = json.loads(msg.payload)
    
    phases = np.array([np.angle(complex(data[1]["aoa"]["iq"][i],data[1]["aoa"]["iq"][i+1])) for i in range(0,len(data[1]["aoa"]["iq"]),2)])
    
    time_model,time_vannila  =[], []
    
    # Model stuff
    time_model.append(time.time())
    iqs = np.array([phases[i:i+4] for i in range(8,len(phases),4)][:-1])
    
    # model_predictions= []
    # for elemnt in iqs:
    #     model_predictions.append(model.predict(elemnt.reshape(1,len(elemnt))))
    global model_predictions
    model_predictions = model.predict(iqs)
    
    moda_model= dicts["backwards_dict"][str(mode(model_predictions).mode[0])]
    moving_avarage_model.append(float(moda_model))
    moving_avarage_model.pop(0) 

    time_model.append(time.time())

    # Vannila stuff
    time_vannila.append(time.time())
    
    aoas = aoa(phases)
    aoas = np.delete(aoas,np.arange(4-1,len(aoas),4))

    media_vannila= np.mean(aoas)
    moving_avarage_vannila.append(media_vannila)
    moving_avarage_vannila.pop(0)

    time_vannila.append(time.time())

    print(f"(Modelo) Moda: {moda_model} \t MA: {np.mean(moving_avarage_model)}")
    print(f"(Vannila) Media: {media_vannila} \t MA: {np.mean(moving_avarage_vannila)}")
    print(f"(Tempo) Modelo: {time_model[1]-time_model[0]} \t Vannila: {time_vannila[1]-time_vannila[0]}\n")


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# client.connect("broker.hivemq.com", 1883, 60)
client.connect("test.mosquitto.org", 1883, 60)

client.loop_forever()

Conectado com o código de resultado 0


C:\Users\pedro\AppData\Local\Temp\ipykernel_4220\1585996196.py:34: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  moda_model= dicts["backwards_dict"][str(mode(model_predictions).mode[0])]


(Modelo) Moda: 30 	 MA: 3.75
(Vannila) Media: 37.93115157438027 	 MA: 4.741393946797534
(Tempo) Modelo: 0.005999565124511719 	 Vannila: 0.001001119613647461

(Modelo) Moda: 40 	 MA: 8.75
(Vannila) Media: 36.31646675226446 	 MA: 9.28095229083059
(Tempo) Modelo: 0.009547948837280273 	 Vannila: 0.000993967056274414

(Modelo) Moda: 40 	 MA: 13.75
(Vannila) Media: 36.104686001184916 	 MA: 13.794038040978705
(Tempo) Modelo: 0.009006738662719727 	 Vannila: 0.0

(Modelo) Moda: 30 	 MA: 17.5
(Vannila) Media: 37.815823965954806 	 MA: 18.521016036723054
(Tempo) Modelo: 0.004503965377807617 	 Vannila: 0.0

(Modelo) Moda: 40 	 MA: 22.5
(Vannila) Media: 36.12657246868899 	 MA: 23.03683759530918
(Tempo) Modelo: 0.0170438289642334 	 Vannila: 0.0

(Modelo) Moda: 40 	 MA: 27.5
(Vannila) Media: 43.96061171620879 	 MA: 28.53191405983528
(Tempo) Modelo: 0.0062487125396728516 	 Vannila: 0.0

(Modelo) Moda: 30 	 MA: 31.25
(Vannila) Media: 37.15851459734707 	 MA: 33.17672838450366
(Tempo) Modelo: 0.0647099018

KeyboardInterrupt: 

ModuleNotFoundError: No module named 'redis'